In [89]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

In [90]:
DATASET_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = 'jeresmc7'
TOKEN = "BQDyDz4ue52IylayJiD31-Szb_I-GotdgReNAYv8u_AtKiJ2e57Oj1-AVW5OgOtF9PtkW7FaXqImNIqtj0lbaKTsag51ZPRikt83VU3QY5mI_Yl0NZ6wjMb0y4-EPEdzHULnn03I4fZx-2ImCvNptZ3XC0UyclfdtTdpnE8z6tLrMbbi8Q"

In [91]:
def check_if_valid_data(df: pd.DataFrame) -> bool:
    #Check if dataframe is empty
    if df.empty:
        print("No songs downloaded. Finishing excecution")
        return False
    
    # Primary Keys Check
    if pd.Series(df["played_at"]).is_unique:
        pass
    else:
        raise Exception("Primary key check is violated")
    
    # Check for nulls
    if df.isnull().values.any():
        raise Exception("Null values found")
        
    # Check all timestamps are of yesterday's date
#     yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
#     yesterday = yesterday.replace(hour=0, second=0, microsecond=0)
    
#     timestamps = df["timestamp"].tolist()
#     for timestamp in timestamps:
#         if datetime.datetime.strptime(timestamp, "%Y-%m-%d") != yesterday:
#             raise Exception("At least one of the returned songs does not come from within the last 24 hours")
            
            

In [92]:
if __name__ == "__main__":

    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : "Bearer {token}".format(token=TOKEN)
    }
    
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = 1484811043508
    
#     int(yesterday.timestamp()) * 1000
    
    print(yesterday_unix_timestamp)
    
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = headers)
    
    data = r.json()

    print(data)

1484811043508
{'items': [{'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/12Chz98pHFMPJEknJQMWvI'}, 'href': 'https://api.spotify.com/v1/artists/12Chz98pHFMPJEknJQMWvI', 'id': '12Chz98pHFMPJEknJQMWvI', 'name': 'Muse', 'type': 'artist', 'uri': 'spotify:artist:12Chz98pHFMPJEknJQMWvI'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY', 'MA', 'MC', 'MD',

In [93]:
song_names = []
artist_names = []
played_at_list = []
timestamps = []

for song in data["items"]:
    song_names.append(song["track"]["name"])
    artist_names.append(song["track"]["album"]["artists"][0]["name"])
    played_at_list.append(song["played_at"])
    timestamps.append(song["played_at"][0:10])
    
song_dict = {
    "song_name" : song_names,
    "artist_name" : artist_names,
    "played_at" : played_at_list,
    "timestamp" : timestamps  
}

song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])

# Validate
if check_if_valid_data(song_df):
    print("Data valid, proceed to Load Stage")

print(song_df)

                                            song_name  \
0                                            Survival   
1                                 La Era del Rivotril   
2                                         En EL Limbo   
3                          Ya No Sé Que Hacer Conmigo   
4                                         This Is War   
5                                  That Funny Feeling   
6                                   Thought Contagion   
7   New Divide - Live from La Caja Mágica, Madrid,...   
8   Waiting for the End - Live from La Caja Mágica...   
9                                         My December   
10                            Supermassive Black Hole   
11                                    Un Poco Perdido   
12                                        Radioactive   
13                                      Crawl Back In   
14                                           Darkside   
15                                   I Am the Highway   
16                             

In [94]:
# Load
engine = sqlalchemy.create_engine(DATASET_LOCATION)
conn = sqlite3.connect('my_played_tracks.sqlite')
cursor = conn.cursor()

sql_query = """
CREATE TABLE IF NOT EXISTS my_played_tracks (
    song_name VARCHAR(200),
    artist_name VARCHAR(200),
    played_at VARCHAR(200),
    timestamp VARCHAR(200),
    CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
)
"""

cursor.execute(sql_query)

In [95]:
# index false porque no queremos tener el indice de pandas
# if exists es porque si la tabla existe, le agregamos la data
try:
    song_df.to_sql("my_played_tracks", engine, index=False, if_exists='append')
except:
    print("Data alredy exists in the database")
    

Data alredy exists in the database


In [97]:
## Reading the data
def readSqliteTable():
    try:
        sqliteConnection = sqlite3.connect('my_played_tracks.sqlite')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_select_query = """SELECT * from my_played_tracks"""
        cursor.execute(sqlite_select_query)
        records = cursor.fetchall()
        print("Total rows are:  ", len(records))
        print("Printing each row")
        for row in records:
            print("song_name: ", row[0])
            print("artist_name: ", row[1])
            print("played_at: ", row[2])
            print("timestamp: ", row[3])
            print("\n")

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to read data from sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")

readSqliteTable()

Connected to SQLite
Total rows are:   20
Printing each row
song_name:  Survival
artist_name:  Muse
played_at:  2022-09-01T14:49:11.054Z
timestamp:  2022-09-01


song_name:  La Era del Rivotril
artist_name:  Alan Sutton y las criaturitas de la ansiedad
played_at:  2022-09-01T14:44:53.512Z
timestamp:  2022-09-01


song_name:  En EL Limbo
artist_name:  La Vela Puerca
played_at:  2022-09-01T14:41:11.020Z
timestamp:  2022-09-01


song_name:  Ya No Sé Que Hacer Conmigo
artist_name:  El Cuarteto De Nos
played_at:  2022-09-01T14:36:28.262Z
timestamp:  2022-09-01


song_name:  This Is War
artist_name:  Thirty Seconds To Mars
played_at:  2022-09-01T14:32:25.761Z
timestamp:  2022-09-01


song_name:  That Funny Feeling
artist_name:  Bo Burnham
played_at:  2022-09-01T14:26:58.184Z
timestamp:  2022-09-01


song_name:  Thought Contagion
artist_name:  Muse
played_at:  2022-09-01T14:21:38.804Z
timestamp:  2022-09-01


song_name:  New Divide - Live from La Caja Mágica, Madrid, Spain, 11/7/2010
artist_na